In [99]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
from nltk.corpus import wordnet 
from nltk.corpus import stopwords
import nltk
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfFile, PdfPages
from ctypes.wintypes import SIZE
from astropy.timeseries import LombScargle
from matplotlib.backends.backend_pdf import PdfFile, PdfPages
import numpy as np
from astropy.timeseries import LombScargle
import matplotlib.transforms as transforms
from matplotlib.offsetbox import AnchoredText
from scipy.signal import argrelextrema
import time 

In [100]:
def average_word_depth(gcs,start,end): #這邊的start,end單位是no. of timeframe
    endtime = end
    df = pd.read_csv(gcs) 
    if end == "end":
        end = len(df) 
    else:
        end = end+10
    df['Start_time'] = pd.to_numeric(df['Start_time'])
    #df['end_time'] = pd.to_numeric(df['end_time'])
    
    words = []
    for index, rows in df.loc[(df["Start_time"]>=start) & (df["Start_time"]<=(end))].iterrows():
        words.append(rows["Words"])
    syn_counts = 0
    num_of_hyponyms_below = 0
    worddepths = []
    for words_num in range(0, len(words)): #標定字
        if len(wordnet.synsets(words[words_num])) > 0 :
            syns1 = wordnet.synsets(words[words_num])
            syns1_num = len(syns1)#這個字有幾個synset
            syn_counts += syns1_num
            
            for synsnum in range(0,syns1_num):  #把每個synset的depth 都加上去
                #synset_depth_sum+=syns1[synsnum].min_depth()
                ### 用來算到lowest hyponyms的距離
                threshhold = []
                threshhold = syns1[synsnum].hyponyms()  
                hyponyms_layer_count = 0
                #print(threshhold)
                #print("word:",syns1[synsnum])
                if threshhold != []:
                    yesorno=True
                else:
                    yesorno = False
                while_count  = 0
                while yesorno:
                    while_count +=1
                    #print("進第",while_count,"層")
                    yesorno = False
                    hyponyms_layer_count += 1
                    new_hyponyms = []
                    #startt = time.time()
                    for syns in threshhold:
                        #print("syns",syns)
                        #print("thresh",threshhold)
                        if syns is not []:
                            yesorno = True 
                            #print("有hypo",syns)
                            new_hyponyms += syns.hyponyms()
                            #print(new_hyponyms)       
                     
                    threshhold = []
                    threshhold = new_hyponyms
                    #print("the",threshhold)
                    new_hyponyms.clear()
                    new_hyponyms = []
               
                if hyponyms_layer_count !=0:
                    num_of_hyponyms_below += hyponyms_layer_count
                else:
                    num_of_hyponyms_below +=0
                #print(hyponyms_layer_count)
        else:
            continue
    print(num_of_hyponyms_below, syn_counts)
    print("start:",start,"end:",endtime, ", hyponyms_layer_count=", (num_of_hyponyms_below/syn_counts))

In [101]:
i=13
average_word_depth(f"data/Text/0{i}(done).csv",0,200)
average_word_depth(f"data/Text/0{i}(done).csv",200,480)

i=18
average_word_depth(f"data/Text/0{i}(done).csv",0,150)
average_word_depth(f"data/Text/0{i}(done).csv",150,480)

i=11
average_word_depth(f"data/Text/0{i}(done).csv",0,300)


2262 2971
start: 0 end: 200 , hyponyms_layer_count= 0.7613598115112756
3104 3808
start: 200 end: 480 , hyponyms_layer_count= 0.8151260504201681
1772 2387
start: 0 end: 150 , hyponyms_layer_count= 0.7423544197737746
4378 6147
start: 150 end: 480 , hyponyms_layer_count= 0.7122173417927444
3870 4768
start: 0 end: 300 , hyponyms_layer_count= 0.8116610738255033


In [102]:
syns1 = wordnet.synsets("college_student")
print(syns1[0].hyponyms())

[Synset('matriculate.n.01'), Synset('student_teacher.n.01')]
